# Create Tool AGI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/CreateTool-AGI/blob/main/examples/createtoolagi.ipynb)

In [1]:
!pip install git+https://github.com/fuyu-quant/CreateTool-AGI.git

  Cloning https://github.com/fuyu-quant/CreateTool-AGI.git to /tmp/pip-req-build-ealeth80
  Running command git clone --filter=blob:none --quiet https://github.com/fuyu-quant/CreateTool-AGI.git /tmp/pip-req-build-ealeth80
  Resolved https://github.com/fuyu-quant/CreateTool-AGI.git to commit d61c1563da74f6f8ea9098b24e05c5b28d6d4621
  Preparing metadata (setup.py) ... done


In [2]:
%%capture
!pip install langchain==0.0.167
!pip install openai==0.27.4

In [3]:
from createtoolagi import CreateToolAGI

import os
#os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

## Normal LLM

In [4]:
from langchain.llms import OpenAI

input = 'What is the sum of the prime numbers from 1 to 100000?'

llm = OpenAI(temperature=0, model_name = 'gpt-4')

llm(input)

'The sum of prime numbers from 1 to 100000 is 454396537.'

## Execute Create Tool AGI

In [5]:
input = 'What is the sum of the prime numbers from 1 to 100000?'
tools = []
ctagi = CreateToolAGI()

ctagi.run(input, tools)

> Generalize the input task.
Generalized Task：What is the sum of the prime numbers from A to B?

> Determine if you should make a tool.
I must create the tool before executing.

try:1
Error occurred: CreateToolAGI._tool_make() takes 3 positional arguments but 4 were given

try:2
Error occurred: CreateToolAGI._tool_make() takes 3 positional arguments but 4 were given

try:3
Error occurred: CreateToolAGI._tool_make() takes 3 positional arguments but 4 were given

try:4
Error occurred: CreateToolAGI._tool_make() takes 3 positional arguments but 4 were given

try:5
Error occurred: CreateToolAGI._tool_make() takes 3 positional arguments but 4 were given

Reached the maximum number of tries.


## Export to file ver

In [12]:
input = 'Sum of prime numbers from 1 to 50'
tools = []
folder_path = '/home/CreateTool-AGI/data/'

ctagi = CreateToolAGI()


ctagi.run(input, tools, folder_path)

> Generalize the input task.
Generalized Task：Sum of prime numbers from A to B

> Determine if you should make a tool.
I must create the tool before executing.

> Create a tool.
Completed!
Created tool name：SumOfPrimesTool



> Entering new AgentExecutor chain...
I need to find the sum of prime numbers between 1 and 50. I can use the SumOfPrimesTool for this.
Action: SumOfPrimesTool
Action Input: 1,50
Observation: 328
Thought:I now know the final answer.
Final Answer: 328

> Finished chain.


## use the tool you created

In [15]:
from SumOfPrimesTool import SumOfPrimesTool

In [16]:
# Change inputs to new ones
input = 'Sum of prime numbers from 8 to 15'

# Use the tool you created
tools = [SumOfPrimesTool()]

ctagi = CreateToolAGI()

ctagi.run(input, tools)

> Generalize the input task.
Generalized Task：Sum of prime numbers from A to B

> Determine if you should make a tool.
I do not need to create a tool to run it.



> Entering new AgentExecutor chain...
I need to find the sum of prime numbers between 8 and 15.
Action: SumOfPrimesTool
Action Input: 8,15
Observation: 24
Thought:I now know the final answer.
Final Answer: 24

> Finished chain.
